## Análisis del Dataset MovieLens

Objetivos del Práctico:
    
    1) Obtener reglas de asociación entre películas.
        derivar reglas de la forma {A} -> {B}
    2) Aplicar diferentes métricas de ordenamiento.
    3) Hacer un pequeño informe.
    

  **Primero se importan las librerias necesarias**

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
from itertools import combinations, groupby
from collections import Counter
from efficient_apriori import apriori

  **En segundo lugar, se leen los dataset y se exploran los datos**

In [2]:
# Sample data
movies = pd.read_csv('ml-20m/movies.csv', encoding = 'utf8')
ratings = pd.read_csv('ml-20m/ratings.csv', encoding = 'utf8')
#links = pd.read_csv('ml-20m/links.csv')
#gen_score = pd.read_csv('ml-20m/genome-scores.csv')
#gen_tags = pd.read_csv('ml-20m/genome-tags.csv')
#tags = pd.read_csv('ml-20m/tags.csv')

In [3]:
print("Movies:")
movies.head()

Movies:


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
movieId    27278 non-null int64
title      27278 non-null object
genres     27278 non-null object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


In [5]:
movies.describe(include='all')

,movieId,title,genres
count,27278.000000,27278,27278
unique,NaN,27262,1342
top,NaN,Chaos (2005),Drama
freq,NaN,2,4520
mean,59855.480570,NaN,NaN
std,44429.314697,NaN,NaN
min,1.000000,NaN,NaN
25%,6931.250000,NaN,NaN
50%,68068.000000,NaN,NaN
75%,100293.250000,NaN,NaN


In [6]:
movies.isnull().any()

movieId    False
title      False
genres     False
dtype: bool

In [7]:
# Separando los años del título
movies['year'] =movies['title'].str.extract('.*\((.*)\).*',expand = False)
movies.head(5)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995


In [8]:
movies.isnull().any()

movieId    False
title      False
genres     False
year        True
dtype: bool

In [9]:
print("Ratings:")
ratings.head()

Ratings:


,userId,movieId,rating,timestamp
0,1,2,3.5,1112486027
1,1,29,3.5,1112484676
2,1,32,3.5,1112484819
3,1,47,3.5,1112484727
4,1,50,3.5,1112484580


In [10]:
del ratings['timestamp']

In [11]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 3 columns):
userId     int64
movieId    int64
rating     float64
dtypes: float64(1), int64(2)
memory usage: 457.8 MB


In [12]:
print("Ratings:")
ratings.head()

Ratings:


,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [13]:
ratings.describe()

,userId,movieId,rating
count,2.000026e+07,2.000026e+07,2.000026e+07
mean,6.904587e+04,9.041567e+03,3.525529e+00
std,4.003863e+04,1.978948e+04,1.051989e+00
min,1.000000e+00,1.000000e+00,5.000000e-01
25%,3.439500e+04,9.020000e+02,3.000000e+00
50%,6.914100e+04,2.167000e+03,3.500000e+00
75%,1.036370e+05,4.770000e+03,4.000000e+00
max,1.384930e+05,1.312620e+05,5.000000e+00


In [14]:
ratings.isnull().any()

userId     False
movieId    False
rating     False
dtype: bool

In [15]:
#here we  make census of the genres:
genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [16]:
genre_labels

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

Es posible hacer las siguientes observaciones:

Dataset Movies:
1. Contiene 27278 datos.
2. El feature "movieID" es un valor único para cada película que se describe en "title" y "genre".
3. El feature "title" contiene el nombre de cada película y el año de estreno en cartelera.
4. El feature "genre" contiene los géneros de la película. Este género no es único, si no que se encuentran muchos nombres adentro.
5. Si no se separan los años del dataset, este no contiene datos nulos.

Dataset Ratings:
1. Contiene 20000263 datos.
2. El feature "userID" indica el número de usuario. En este dataset, se poseen todos los puntajes en "rating" que los diferentes usuarios "userID" le pusieron a cada película "movieID". Además se observa un feature "timestamp" que indica cuándo se hizo la evaluación.
3. El dataset no contiene datos nulos.
4. Algunos géneros fueron mal colocados como "IMAX" o no han sido clasificadas.
5. Se elimina el feature "timestamp" debido a que se considera que no aporta información significativa para hacer reglas de asociación.

  **En tercer lugar, se busca hacer reglas de asociación**

En esta instancia, se aplica el índice zscore sobre el rating en función de la media y desvio de las puntuaciones puestas por cada usuario, debido a que las puntuaciones tienen una carga subjetiva asociada a cada usuario, el modo de evaluar las peliculas. Una vez estandarizado el rating, se filtra con el rating por encima de 1, ya que la película se encuentra por arriba de la media en una desviación. De este modo, el dataset se ve disminuido considerablemente y aumentan las chances de recomendar una película que le pueda gustar al usuario.

In [17]:
# calculo un zscore por usuario
medias = ratings['rating'].groupby(ratings.userId).mean()
desviaciones = ratings['rating'].groupby(ratings.userId).std()

In [18]:
ratings['medias'] = ratings['userId'].apply(lambda userid: medias[userid])

In [19]:
ratings['desviaciones'] = ratings['userId'].apply(lambda userid: desviaciones[userid])

In [20]:
ratings['ratingNorm'] = (ratings.rating-ratings.medias)/ratings.desviaciones

In [21]:
ratings.head()

,userId,movieId,rating,medias,desviaciones,ratingNorm
0,1,2,3.5,3.742857,0.382284,-0.635279
1,1,29,3.5,3.742857,0.382284,-0.635279
2,1,32,3.5,3.742857,0.382284,-0.635279
3,1,47,3.5,3.742857,0.382284,-0.635279
4,1,50,3.5,3.742857,0.382284,-0.635279


In [22]:
sample = ratings[ratings.ratingNorm > 1]
sample.head()

,userId,movieId,rating,medias,desviaciones,ratingNorm
30,1,1196,4.5,3.742857,0.382284,1.980576
31,1,1198,4.5,3.742857,0.382284,1.980576
131,1,4993,5.0,3.742857,0.382284,3.288503
142,1,5952,5.0,3.742857,0.382284,3.288503
158,1,7153,5.0,3.742857,0.382284,3.288503


In [23]:
#decodificar el nombre de las peliculas
movies_df = pd.merge(sample[['userId','movieId']], movies[['movieId','title']] ,on='movieId', how= "inner")

display(movies_df.head(20))

,userId,movieId,title
0,1,1196,Star Wars: Episode V - The Empire Strikes Back...
1,7,1196,Star Wars: Episode V - The Empire Strikes Back...
2,21,1196,Star Wars: Episode V - The Empire Strikes Back...
3,24,1196,Star Wars: Episode V - The Empire Strikes Back...
4,31,1196,Star Wars: Episode V - The Empire Strikes Back...
5,51,1196,Star Wars: Episode V - The Empire Strikes Back...
6,55,1196,Star Wars: Episode V - The Empire Strikes Back...
7,56,1196,Star Wars: Episode V - The Empire Strikes Back...
8,69,1196,Star Wars: Episode V - The Empire Strikes Back...
9,77,1196,Star Wars: Episode V - The Empire Strikes Back...


In [24]:
movies_df = movies_df.sort_values( by='userId', axis=0, ascending=True, 
                                    inplace=False, kind='quicksort', 
                                    na_position='last')
print(movies_df)

         userId  movieId                                              title
0             1     1196  Star Wars: Episode V - The Empire Strikes Back...
16285         1     1198  Raiders of the Lost Ark (Indiana Jones and the...
31101         1     4993  Lord of the Rings: The Fellowship of the Ring,...
44980         1     5952      Lord of the Rings: The Two Towers, The (2002)
68553         1     8636                                Spider-Man 2 (2004)
...         ...      ...                                                ...
3094414  138493     5720  Friend Is a Treasure, A (Chi Trova Un Amico, T...
2815895  138493    47465                                    Tideland (2005)
2589031  138493     2843  Black Cat, White Cat (Crna macka, beli macor) ...
2387478  138493     2787                                   Cat's Eye (1985)
1324475  138493     3949                         Requiem for a Dream (2000)

[3095998 rows x 3 columns]


In [25]:
ratings_2 = movies_df.values[:,[0,2]]
print(ratings_2)

[[1 'Star Wars: Episode V - The Empire Strikes Back (1980)']
 [1
  'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)']
 [1 'Lord of the Rings: The Fellowship of the Ring, The (2001)']
 ...
 [138493 'Black Cat, White Cat (Crna macka, beli macor) (1998)']
 [138493 "Cat's Eye (1985)"]
 [138493 'Requiem for a Dream (2000)']]


In [26]:
def transactions_generator():
    for userId, movie_object in groupby(ratings_2, lambda x: x[0]):
        yield [item[1] for item in movie_object]

In [27]:
trans_gen = transactions_generator()

itemsets, rules = apriori(trans_gen, min_support=0.05,  min_confidence=0.5)

rules=sorted(rules, key=lambda rule: rule.confidence)
for rule in rules:
    print(rule) # Prints the rule and its confidence, support, lift, ...

{Godfather, The (1972)} -> {Godfather: Part II, The (1974)} (conf: 0.501, supp: 0.081, lift: 5.339, conv: 1.817)
{Braveheart (1995)} -> {Shawshank Redemption, The (1994)} (conf: 0.504, supp: 0.076, lift: 1.863, conv: 1.471)
{Usual Suspects, The (1995)} -> {Pulp Fiction (1994)} (conf: 0.511, supp: 0.089, lift: 2.160, conv: 1.562)
{Schindler's List (1993)} -> {Shawshank Redemption, The (1994)} (conf: 0.514, supp: 0.094, lift: 1.899, conv: 1.501)
{Star Wars: Episode V - The Empire Strikes Back (1980)} -> {Star Wars: Episode VI - Return of the Jedi (1983)} (conf: 0.522, supp: 0.072, lift: 4.752, conv: 1.862)
{Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)} -> {Star Wars: Episode V - The Empire Strikes Back (1980)} (conf: 0.526, supp: 0.066, lift: 3.803, conv: 1.819)
{Usual Suspects, The (1995)} -> {Shawshank Redemption, The (1994)} (conf: 0.529, supp: 0.092, lift: 1.951, conv: 1.547)
{Seven (a.k.a. Se7en) (1995)} -> {Shawshank Redemption, The (1994)} (conf: 

## Informe:
### Reglas de Asociación
Para hacer las reglas de asociación se utilizará el método APRIORI utilizando los siguientes features:

Transacciones: "userId"
Items: "movieId" y "title"

Se considera como transacción al usuario que ha evaluado a una serie de películas. Por lo tanto, estas películas evaluadas serán los items. 

https://pypi.org/project/efficient-apriori/

Como tenemos muchos datos, para almacenar las transacciones debemos usar un generador de listas en lugar de una lista de listas. El generador que defino a continuación va creando cada transacción de a una para no usar toda la memoria.


### Métricas

El $soporte$ de un conjunto de items $X$ en una base de datos $D$ se define como la proporción de transacciones en la base de datos que contiene dicho conjunto de items: $$ Sop(X) = \dfrac{|X|}{|D|} $$

Un valor alto de soporte indica que la regla se encuentra en muchas transacciones. Por otro lado, una regla con soporte bajo, podría ser casualidad.

La confianza de esta regla se define como: $$ Conf(X \Rightarrow Y) = \dfrac{Sop(X \cap Y)}{Sop(X)} = \dfrac{|X \cap Y|}{|X|}$$

Es decir, el porcentaje de transacciones que contienen $Y$, entre las transacciones que contienen $X$. El significado de un mayor valor de confianza indica mayor probabilidad de que la regla sea cierta para una transacción. Si una regla tiene baja confianza, es probable que no exista relación entre antecedente y consecuente.

Cabe aclarar que, tanto el soporte como la confianza son probabilidades, por lo tanto sus valores varían entre 0 y 1.

El indicador lift expresa cuál es la proporción del soporte observado de un conjunto de productos respecto del soporte teórico de ese conjunto dado el supuesto de independencia. El lift está dado por las siguientes fórmulas equivalentes:

$$ Lift =   \dfrac{Sop(X \Rightarrow Y)}{Sop(X) \times Sop(Y)} = \dfrac{P(X \cup Y)}{P(X) \times P(Y)} = \dfrac{N \times frec(X,Y)}{frec(X) \times frec(Y)}$$

Un valor de lift = 1 indica que ese conjunto aparece una cantidad de veces acorde a lo esperado bajo condiciones de independencia. Un valor de lift > 1 indica que ese conjunto aparece una cantidad de veces superior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos se encuentren en el conjunto más veces de lo normal). Un valor de lift < 1 indica que ese conjunto aparece una cantidad de veces inferior a lo esperado bajo condiciones de independencia (por lo que se puede intuir que existe una relación que hace que los productos no estén formando parte del mismo conjunto más veces de lo normal).

La convicción expresa que tan independientes son las variables $X$ y $¬Y$,

$$ Conv = \dfrac{1 - Sop(Y)}{1 - Conf(X \Rightarrow Y)} = \dfrac{frec(X) \times frec(¬Y)}{frec(X, ¬Y)} $$
donde $frec(¬Y)$ es la cantidad de transacciones en la base de datos que no contienen a $Y$ y $frec(X, ¬Y)$ es la cantidad de transacciones en la base de datos que contienen a $X$ y no contienen a $Y$.

La convicción va de 1 a infinito (si la confianza es 1, la convicción es infinita, no 0). Más convicción indica mayor grado de implicación. Altos valores para convicción (cuando $frec(X, ¬Y)$ tiende a cero) afirman la convicción de que esta regla representa una causalidad.

### Análisis de Resultados

A primera vista, al observar las películas sugeridas por el algoritmo, es posible observar que las reglas generadas tienen sentido. Como ejemplo, las películas que pertenecen a una misma cronología son sugeridas. Del mismo modo, las reglas relacionan películas de un mismo director (por ejemplo: {Reservoir Dogs (1992)} -> {Pulp Fiction (1994)}).

Las reglas obtenidas están ordenadas según su confianza de menor a mayor, es decir que las primeras reglas son menos confiables que las últimas. La menor confianza obtenida fue de $0.501$, mientras que la mayor de $0.892$.

Al analizar los soportes, se observa que las reglas derivadas tienen valores muy bajos. En general, el orden de magnitud resulta menor a $0.1$, indicando que las reglas de asociación se encuentran en pocas transacciones de la base de datos.

Los valores de lift son mayores a 1 en todos los casos. Esto significa que, para cada regla $\{X \} \Rightarrow \{Y \}$ derivada, $X$ e $Y$ aparecen juntos una cantidad de veces superior a lo esperado bajo condiciones de independencia, indicando que la aparición de $X$ tiene un efecto positivo sobre la aparición de $Y$.

Los valores de convicción también son altos en todos los casos, indicando causalidad entre $X$ e $Y$.

### Conclusiones

- El algoritmo Apriori es simple e intuitivo para obtener reglas de asociación de manera rápida y eficaz.
- La base de datos tuvo que ser estandarizada para filtrar subjetividades propias de los usuarios. Además, se filtró la base de datos para trabajar solo con los valores mejores puntuados. Esto impactó en las recomendaciones al observarse que fuera de las recomentaciones entre películas de las mismas sacas, se recomiendan filmes clásicos taquilleros.
- Los valores de soporte resultaron bajos indicando que las reglas de asociación fueron encontradas en pocas transacciones. Sin embargo, la confianza asociada a estas reglas resultaron elevadas (hasta 89%).
- Por último, los valores de lift y convicción indican en general que las reglas obtenidas aparecen mayor veces de las esperadas y con un importante rasgo de causalidad.